# 주가지수만들기
--------------
<STEP 1>
   1. TCI 지수 선정 : 일일 확진자(과거데이터 가중치{EMA,MA등 사용}, 거리두기 단계(1~4단계), Additional Wieghted_Value : 누적사망자/누적확진자)
   > (누적사망자/누적확진자) 가 클수록 코로나의 위험도가 증가한다고 판단하여 가중치 값으로 선정
   2. TCI 지수와 코스피, 코스닥 움직임 상관관계 예측
---------------
<STEP 2>
   1. TCI 지수 산출을 위한 데이터 만들기 ('2020.03.10' ~ '2022.02.28')
   > 지수 산출방법 : ```(추가로더적을 부분)``` 4번까지 한다음에 다시 피드백해서 정하기
   2. 코스피, 코스닥 동기간 종가 불러오기
   3. 임의로 만든 TCI와 KOSPI & KOSDAQ 상관관계 비교
   4. 산업군별로 데이터분석
      > 상관성 높은 산업군에서 예측모델 사용권장
      ``예측은 아마도 헬스케어는 + , 운송업은 -``

In [20]:
import pandas as pd
import numpy as np
import FinanceDataReader as fdr
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

In [21]:
# Assign Corona.csv to df
df = pd.read_csv('./datasets/Corona.csv')
dist = pd.read_excel('./datasets/Dist.xlsx')

# Preprocessing Corona Data

In [22]:
# Preprocessing Corona Data
df.columns = ["1", 'Date', 'DeadCnt', 'ConfirmedCnt', "2", "3", "4", "5", "6", "7"]
dist.columns = ['Date', 'Level']
df.drop([str(i) for i in range(1,8)],axis=1,inplace=True) # Drop Unecessary Columns
df = df.sort_values('Date') # Sorting DATE
df['DailyDeadCnt'] = (df['DeadCnt']-df['DeadCnt'].shift(1)).fillna(0) # Create DailyDeadCnt by using difference & fill NAN by 0
df['DailyConfirmedCnt'] = (df['ConfirmedCnt']-df['ConfirmedCnt'].shift(1)).fillna(0) # Create DailyConfirmedCnt by using difference & fill NAN by 0
df['D/C(%)'] = (df['DeadCnt']/df['ConfirmedCnt'])*100 # Create D/C
df.drop(['DeadCnt','ConfirmedCnt'],axis=1,inplace=True) # Drop Unecessary Columns
df.reset_index(drop=True, inplace=True) # Reset index
df['Date'] = df['Date'].str[0:4] + "-" + df['Date'].str[5:7] + "-" +df['Date'].str[8:10] # Change Date style YYYY-MM-DD
df['Date'] = pd.to_datetime(df['Date'],infer_datetime_format=True) # Change datatype from object to datetime
df = pd.merge(df, dist, left_on='Date', right_on='Date', how = 'left') # Merge df & Corona Level
df.fillna(3, inplace=True) # ALL NAN index in Level column fill 3


# Preprocessing stock index

In [23]:
# Read Stock Index 
Kospi = fdr.DataReader('KS11') 
Kodaq = fdr.DataReader('KQ11')

In [24]:
# Preprocessing
df_c = df
sp_str = ['Kospi', 'Kosdq']
sp_df = [Kospi, Kodaq]

for i, col in enumerate(sp_df): # Merge Kospi & Kodaq into df_c
    corona_df = pd.merge(df_c, col['Close'], left_on='Date', right_on=col.index, how='inner')
    corona_df.rename(columns={'Close': sp_str[i]}, inplace=True)
    df_c = corona_df


In [25]:
# Check corona_df to finish preprocessing 
corona_df

,Date,DailyDeadCnt,DailyConfirmedCnt,D/C(%),Level,Kospi,Kosdq
0,2020-03-10,0.0,0.0,0.718754,3.0,1962.93,619.97
1,2020-03-11,6.0,242.0,0.773694,3.0,1908.27,595.61
2,2020-03-12,6.0,114.0,0.838734,3.0,1834.33,563.49
3,2020-03-13,1.0,110.0,0.839704,3.0,1771.44,524.00
4,2020-03-16,0.0,74.0,0.910636,3.0,1714.86,504.51
...,...,...,...,...,...,...,...
481,2022-02-22,58.0,99569.0,0.347958,2.0,2706.79,868.11
482,2022-02-23,99.0,171442.0,0.326597,2.0,2719.53,877.33
483,2022-02-24,82.0,170015.0,0.307660,2.0,2648.80,848.21
484,2022-02-25,94.0,165887.0,0.292037,2.0,2676.76,872.98
